# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
if 'uk_food' in mongo.list_database_names():
    print("The 'uk_food' database exists.")
else:
    print("The 'uk_food' database does not exist.")

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
collection_names = db.list_collection_names()
for collection_name in collection_names:
    print(collection_name)

In [ ]:
# review the collections in our new database
collection_names = db.list_collection_names()
for collection_name in collection_names:
    print(collection_name)

In [ ]:
# review a document in the establishments collection
collection_name = 'establishments'
document = db[collection_name].find_one()
print(document)


In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    'name': 'Penang Flavours',
    'cuisine': 'Halal',
    'location': 'Greenwich',
    'rating': None  }
collection_name = 'establishments'


In [ ]:
# Insert the new restaurant into the collection
result = db[collection_name].insert_one(new_restaurant)


In [ ]:
# Check that the new restaurant was inserted
if result.inserted_id:
    print(f"New restaurant '{new_restaurant['name']}' with ID {result.inserted_id} has been added to the database.")
else:
    print("Failed to insert the new restaurant into the database.")
    

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_query = {
    'BusinessType': 'Restaurant/Cafe/Canteen'
}
business_type_data = db['businesses'].find_one(business_type_query, {'BusinessTypeID': 1, 'BusinessType': 1})


3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
if business_type_data:
    business_type_id = business_type_data['BusinessTypeID']
    new_restaurant_query = {'name': 'Penang Flavours'}
    update_query = {'$set': {'BusinessTypeID': business_type_id}}
    update_result = db['establishments'].update_one(new_restaurant_query, update_query)
    


In [ ]:
# Confirm that the new restaurant was updated
    if update_result.modified_count > 0:
        print(f"The new restaurant 'Penang Flavours' has been updated with BusinessTypeID {business_type_id}.")
    else:
        print("The new restaurant was not updated.")
else:
    print("BusinessTypeID for 'Restaurant/Cafe/Canteen' was not found.")


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_query = {
    'LocalAuthorityName': 'Dover'
}
dover_count = db['establishments'].count_documents(dover_query)
print(f"Number of documents with LocalAuthorityName 'Dover': {dover_count}")


In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = db['establishments'].delete_many(dover_query)


In [ ]:
# Check if any remaining documents include Dover
remaining_dover_count = db['establishments'].count_documents(dover_query)
print(f"Number of remaining documents with LocalAuthorityName 'Dover': {remaining_dover_count}")


In [ ]:
# Check that other documents remain with 'find_one'
sample_document = db['establishments'].find_one()
print("Sample document (after deletion):")
print(sample_document)

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
from decimal import Decimal
db['establishments'].update_many({}, [
    {
        '$set': {
            'longitude': {
                '$convert': {
                    'input': '$longitude',
                    'to': 'decimal',
                    'onError': None 
                }
            },
            'latitude': {
                '$convert': {
                    'input': '$latitude',
                    'to': 'decimal',
                    'onError': None  
                }
            }
        }
    }
])


Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
db['establishments'].update_many(
    {'RatingValue': {'$nin': [1, 2, 3, 4, 5]}}, 
    {'$set': {'RatingValue': None}}  
)


In [ ]:
# Check that the coordinates and rating value are now numbers
sample_document = db['establishments'].find_one()
print("Sample document (after data type conversion):")
print(sample_document)